# 4枚だけを使って計算する
## C++を導入し、高速に
とりあえず作れるのは1枚だけ。


## 準備

In [1]:
# インポート系
!pip install pillow
from PIL import Image, ImageDraw, ImageFont
from collections import defaultdict
from sys import setrecursionlimit
from copy import deepcopy
import time
from functools import lru_cache
import sys
from IPython.utils.py3compat import u_format
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import imageio
import os
import csv
import subprocess

# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# 準備
setrecursionlimit(10**8)  # 再帰計算の上限回数を上げる
def LL(h,w,e=0): return [[e for i in range(w)] for j in range(h)]
def LLL(a,b,c,e=0): return [LL(b,c,e) for _ in range(a)]
def LLLL(a,b,c,d,e=0): return [LLL(b,c,d,e) for _ in range(a)]

Mounted at /content/drive


In [20]:
# 変数名定義など

p = 5  # 禁止手数
s = 1010  # 探索範囲
k = 1000  # 描画するところ
ver = 8
M = 9  # 登場する色の数

csv_path = "/content/drive/MyDrive/result_sbnim.csv"  # csvファイル
png_dir = '/content/drive/MyDrive/研究 組合せ/３次元スライスp={}, s={} ({})/'.format(p,s,ver)  # pngファイルの置き場所
gif_filename_1 = 'output_part1.gif'
gif_filename_2 = 'output_part2.gif'
gif_filename_final = 'p={}.gif'.format(p)

colormap = plt.get_cmap('hsv')  # カラーマップ
width, height = 6400, 4800
margin = 50  # 余白のサイズ

# 画像にステータスを書き込む
font_path = '/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf'
font_size = 60
status_position = (2600,100)
font = ImageFont.truetype(font_path, font_size)

# 該当フォルダがない場合、新しくフォルダを作成する
if not os.path.exists(png_dir):
    os.makedirs(png_dir)

## C++による計算
pとsの値を変更する

In [21]:
%%writefile /content/drive/MyDrive/calc_sbnim.cpp

#include <bits/stdc++.h>
using namespace std;

template<class T> inline bool chmax(T& a, T b) { if (a < b) { a = b; return 1; } return 0; }
template<class T> inline bool chmin(T& a, T b) { if (a > b) { a = b; return 1; } return 0; }
typedef long long ll;
#define rep(i,n) for(int i=0;i<(int)(n);i++)
#define rep1(i,n) for(int i=1;i<(int)(n);i++)
#define replr(i,l,r) for(int i=l;i<(int)(r);i++)
#define All(c) (c).begin(), (c).end()
#define Print(x) cout<<(x)<<endl
#define Prints(x) cout<<setprecision(10)<<(x)<<endl
#define Printl(A,n) rep(i,n) {cout<<A[i]<<' ';} cout<<endl;


int p;  // 禁止手数
int s;  // 見る範囲
int c;  // 描画するところのk
int mf;
int Mf;
int cnt;
int f2[3010][3010];  // 2次元分の初期値
int f3[4][3010][3010];  // dp配列
int d[100];  // p<=80ぐらいでやるならこれでよい

int g2(int x, int y) {
    if (f2[x][y] <= p) return 1;
    return 0;
}

int g3(int t, int x, int y) {
    if (f3[t][x][y] <= p) return 1;
    return 0;
}


int main() {

    ofstream outfile("/content/drive/MyDrive/result_sbnim.csv");

    p = 5;  // 禁止手数
    s = 1010;  // 探索範囲
    c = 1000;  // 画像を取得する場所
    cnt = 0;  // 異常値は100個まで見る
    mf = 1000;  // レンジの最小値
    Mf = -1000;  // レンジの最大値
    set<tuple<int, vector<int>, vector<int>, int>> S;


    /*
    // もとのやつ
    // 1次元
    rep(x, s) {
        f2[x][0] = min(x, p+1);
        f2[0][x] = min(x, p+1);
    }

    // 2次元
    rep1(x, s) {
        rep1(y,s) {
            f2[x][y] = f2[x-1][y] + f2[x][y-1] - f2[x-1][y-1]
                        + g2(x-1,y) + g2(x,y-1) -2*g2(x-1,y-1);
        }
    }
    */

    /*
    // 変更点：急に5の初期値になる
    rep(x, s) {
        if (x >= 50) {
            f2[x][x+4] = p;
            f2[x][x-4] = p;
        }
    }
    */

    // 変更：最初全部1で途中から5の初期値を与える
    rep(x,s) {
        rep(y,s) {
            f2[x][y] = p+1;
        }
    }
    rep(x,s) {
        if (x>=50) {
            f2[x][x+4]=p;
            f2[x][x+2]=p;
            f2[x][x]=p;
            f2[x][x-2]=p;
            f2[x][x-4]=p;
        }
    }




    // 3次元の最初の3枚
    f3[0][0][0] = f2[0][0];

    f3[1][0][0] = f2[1][0];
    f3[1][1][0] = f2[1][0];
    f3[1][0][1] = f2[1][0];

    f3[2][0][0] = f2[2][0];
    f3[2][2][0] = f2[2][0];
    f3[2][0][2] = f2[2][0];

    f3[2][1][1] = f2[1][1];
    f3[2][1][0] = f2[1][1];
    f3[2][0][1] = f2[1][1];

    // それ以降
    for (int k=3; k<s; k++) {

        // 進捗
        if (k%200==0) {
            cout << k << endl;
        }

        // 実質2次元の場所を埋める
        rep(x,k+1) {
            int y = k-x;
            int z = 0;
            f3[3][x][y] = f2[x][y];
            f3[3][x][z] = f2[x][y];
            f3[3][z][y] = f2[x][y];
        }

        // 漸化式に従ってつくる
        rep1(x,k) {
            rep1(y,k) {
                if (x+y>=k) continue;
                // int z = k-x-y;
                f3[3][x][y] = f3[2][x-1][y] + f3[2][x][y-1] + f3[2][x][y]
                                - (f3[1][x][y-1] + f3[1][x-1][y] + f3[1][x-1][y-1])
                                + f3[0][x-1][y-1]
                                + g3(2,x,y) + g3(2,x,y-1) + g3(2,x-1,y)
                                -2 *(g3(1,x,y-1) + g3(1,x-1,y) + g3(1,x-1,y-1))
                                +3 * g3(0,x-1,y-1);
                if (k>=p) {
                    chmin(mf, f3[3][x][y]);
                    chmax(Mf, f3[3][x][y]);
                }


                // 例外的な場所についてチェック
                if (k>=p && (f3[3][x][y]<p-3 || p+4<f3[3][x][y]) && cnt<100) {
                    cout << k << " " << x << " " << y << " " << f3[3][x][y] << endl;
                    cnt++;
                }

                /*
                // 例外だけパターンを列挙する
                if (k>=p && (f3[3][x][y]<=p-3 || p+4<=f3[3][x][y])) {
                    vector<int> V1 = {f3[1][x][y-1], f3[1][x-1][y], f3[1][x-1][y-1]};
                    sort(V1.begin(), V1.end());
                    vector<int> V2 = {f3[2][x-1][y], f3[2][x][y-1], f3[2][x][y]};
                    sort(V2.begin(), V2.end());
                    S.insert(std::make_tuple(f3[0][x-1][y-1], V1, V2, f3[3][x][y]));
                }
                */
            }
        }

        // 必要なところだけテキストに起こす
        if (k==c) {
            rep(x,s) {
                rep(y,s) {
                    outfile << f3[3][x][y] << " ";
                }
                outfile << endl;
            }
        }

        // 次のに向けて送る
        rep(t,3) {
            rep(x,s) {
                rep(y,s) {
                    f3[t][x][y] = f3[t+1][x][y];
                }
            }
        }
        rep(x,s) {
            rep(y,s) {
                f3[3][x][y] = 0;
            }
        }

    }
    outfile.close();

    cout<<mf<<" "<<Mf<<endl;

    /*

    for (const auto& item : S) {
    // itemはタプルです。各要素をアクセスして出力する必要があります。
    // std::get<> を使ってタプルの各要素にアクセスします。

    // 1つ目の要素を出力
    std::cout << std::get<0>(item) << " ";

    // 2つ目の要素（ベクター）を出力
    const auto& vec1 = std::get<1>(item);
    for (const auto& val : vec1) {
        std::cout << val << " ";
    }
    std::cout << " ";

    // 3つ目の要素（ベクター）を出力
    const auto& vec2 = std::get<2>(item);
    for (const auto& val : vec2) {
        std::cout << val << " ";
    }
    std::cout << " ";

    // 4つ目の要素を出力
    std::cout << std::get<3>(item) << std::endl;

}
*/
}



Overwriting /content/drive/MyDrive/calc_sbnim.cpp


In [22]:
!g++ -mcmodel=large -o /content/drive/MyDrive/calc_sbnim /content/drive/MyDrive/calc_sbnim.cpp

In [23]:
!/content/drive/MyDrive/calc_sbnim

200
218 58 102 10
223 58 107 10
224 58 108 10
225 58 109 10
226 58 110 10
226 110 58 10
227 111 58 10
229 58 113 10
232 58 116 10
232 116 58 10
233 58 117 10
234 58 118 10
235 58 119 10
235 119 58 10
236 58 120 10
239 58 123 10
400
600
800
1000
2 10


## Pythonにデータを渡す

In [25]:
f = []

with open(csv_path, "r", newline="", encoding="utf-8") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=" ")
    for row in csvreader:
        a = []
        for i in filter(None, row):
            a.append(int(i))
        f.append(a)

In [26]:
Xm, XM, Ym, YM = 10000,0,10000,0
# 巨大な場合
#width, height = 25600, 19200
#M = 8

# 描画用
Color = []
for i in range(M):
    Color.append(tuple(int(255 * c) for c in colormap(i/M)[:3]))
ps = 1

image = Image.new("RGB", (width, height), "white")
draw = ImageDraw.Draw(image)
points = []

for x in range(k+1):
    for y in range(k+1):
        if x+y>k: continue
        z = k-x-y
        X = x * (3**.5/6 + 1/2) + y * (3**.5/6 - 1/2) - 3**.5/3 * z
        Y = x * (3**.5/6 - 1/2) + y * (3**.5/6 + 1/2) - 3**.5/3 * z
        Xm = min(Xm, X)
        XM = max(XM, X)
        Ym = min(Ym, Y)
        YM = max(YM, Y)
        if f[x][y] == p+1:
            continue
        points.append((X,Y,f[x][y]))

Scale = min((width - 2 * margin) / (XM - Xm), (height - 2 * margin) / (YM - Ym))
# ステータス情報を追記
status = "p={}, k={}, s={}".format(p,k,s)
draw.text(status_position, status, fill=(0,0,0), font=font)

for u,v,i in points:
    plot_x = Scale * (u - Xm) + margin
    plot_y = Scale * (YM - v) + margin
    rgb_color =  Color[i-(p-3)]
    draw.ellipse((plot_x - ps, plot_y - ps, plot_x + ps, plot_y + ps), fill=rgb_color)
image.save(png_dir + "p={}k={}s={}.png".format(p,"{:03}".format(k),s))

In [ ]:
# 蛇足　0の場合の3D図を作成

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Setting the size of the space
size = 500
x_plot, y_plot, z_plot=[],[],[]

for x in range(size):
    for y in range(size):
        for z in range(size):
            if x^y^z==0:
                x_plot.append(x)
                y_plot.append(y)
                z_plot.append(z)

# Plotting
fig = plt.figure(figsize=(100, 80))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_plot, y_plot, z_plot, s=1, c='blue', alpha=0.5)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('3D Plot of Points where x^y^z=0 (XOR) in [0,500] Space')

plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
size = 200
x_plot, y_plot, z_plot=[],[],[]

# 準備
def LL(h,w,e=0): return [[e for i in range(w)] for j in range(h)]
def LLL(a,b,c,e=0): return [LL(b,c,e) for _ in range(a)]
def LLLL(a,b,c,d,e=0): return [LLL(b,c,d,e) for _ in range(a)]

w = LLL(size,size,size)

def put(x,y,z):
    for i in range(x+1,size):
        w[i][y][z] += 1
    for i in range(y+1,size):
        w[x][i][z] += 1
    for i in range(z+1,size):
        w[x][y][i] += 1

for x in range(size):
    for y in range(size):
        for z in range(size):
            if w[x][y][z] == 0:
                put(x,y,z)
            elif w[x][y][z] == 3:
                x_plot.append(x)
                y_plot.append(y)
                z_plot.append(z)

# Plotting
fig = plt.figure(figsize=(100, 80))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_plot, y_plot, z_plot, s=1, c='blue', alpha=0.5)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('3D Plot of Points where w[x][y][z]==3 in [0,500] Space')

plt.show()


Output hidden; open in https://colab.research.google.com to view.